In [87]:
from typing import Tuple
import pandas as pd
import numpy as np
import random

In [88]:
seeds = pd.read_csv("seeds.csv", sep=",")

In [89]:
class ProcessingData:
    # zwracam dataframe w losowej kolejności i resetuje indeksy
    @staticmethod
    def shuffleDF(df: pd.DataFrame) -> pd.DataFrame:
        return df.iloc[random.sample(range(len(df)), len(df))].reset_index(drop=True)

    # normalizuje każdą kolumnę w dataframe (według przekazanej listy) metodą min max
    @staticmethod
    def normalizeDF(df: pd.DataFrame, columnNames: list) -> pd.DataFrame:
        for columnName in columnNames:
            df[columnName] = (df[columnName]-df[columnName].min())/(df[columnName].max()-df[columnName].min())
        return df

    # zwracam wiersze gdzie index jest mniejszy od długość df*wielkość df i wiersze gdzie index jest równy lub większy od długość *wielkość df 
    @staticmethod
    def splitDF(df: pd.DataFrame, trainSize: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        return df[df.index < int(len(df)*trainSize)], df[df.index >= int(len(df)*trainSize)]

    @staticmethod
    def processData(df: pd.DataFrame, columnNames: list, trainSize: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        df = ProcessingData.shuffleDF(df)
        df = ProcessingData.normalizeDF(df, columnNames)
        return ProcessingData.splitDF(df, trainSize)
    

In [103]:
class NaiveBayes:
    # przygotowuje dane potrzebne do obliczeń takie jak prawdopodobieństwo średnia i odchylenie standardowe dla wszystkich typów
    def __init__(self, df: pd.DataFrame) -> None:
        self.dataTypes = df.iloc[:, -1].unique()
        self.dataSplitted = {k : df[df["Type"] == k].iloc[:, :-1] for k in self.dataTypes}
        self.columnNames = df.columns[:-1]
        self.probDict = {k : len(self.dataSplitted[k])/len(df) for k in self.dataTypes}
        self.meanDict = {k : self.dataSplitted[k].mean() for k in self.dataTypes}
        self.stdDict = {k : self.dataSplitted[k].std() for k in self.dataTypes}
    
    # obliczam według wzoru podanego w zadaniu    
    @staticmethod
    def gauss(x, classMean, classStd) -> float:
        if x < classMean - classStd*6**(1/2):
            return 0
        elif x <= classMean:
            return (x-classMean)/(6*(classStd**2)) + 1/((6**(1/2)) * classStd)
        elif x <= classMean + classStd**(1/2):
            return -(x-classMean)/(6*(classStd**2)) + 1/((6**(1/2)) * classStd)
        else: 
            return 0

    # liczę prawdopodbieństwo każej klasy dla przekazanej próbki 
    def predict(self, sample: pd.core.series):
        result = {}
        for d in self.dataTypes:
            prob = self.probDict[d]
            for column in range(len(self.columnNames)):
                prob *= self.gauss(sample[column], self.meanDict[d][column], self.stdDict[d][column])
            result[d] = prob
        return max(result, key=result.get)
    
    #obliczam wynik    
    def score(self, X_test: pd.DataFrame):
        good, bad = 0, 0
        for sample in X_test.values:
            #porównuje wynik z oczekiwanym
            if (x:=self.predict(sample)) == sample[-1]:
                good += 1
            else:
                bad +=1
        return good/(bad+good)*100

In [109]:
# z normalizacja
train_X, test_X = ProcessingData.processData(seeds, seeds.columns[:-1], 0.8)
bayes = NaiveBayes(train_X)
print(bayes.score(test_X))


95.0


In [120]:
# bez normalizacji
seeds = pd.read_csv("seeds.csv", sep=",")
seeds = ProcessingData.shuffleDF(seeds)
train_X, test_X = ProcessingData.splitDF(seeds, 0.8)
bayes = NaiveBayes(train_X)
print(bayes.score(test_X))


85.0
